### STEP 1: Extraction 

#### 1. Importing Necessary Libraries

In [7]:
import requests
import pandas as pd
import psycopg2
import json

#### 2. Carry out extraction

In [4]:
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

In [5]:
querystring = {"limit":"500"}

headers = {
	"x-rapidapi-key": "60e9afa955msh7936cf92edeb77ap10b4a1jsnbfc9face381e",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}
response = requests.get(url, headers=headers, params=querystring)

In [6]:
print(response.json())

{'messages': 'The API is unreachable, please contact the API provider', 'info': 'Your Client (working) ---> Gateway (working) ---> API (not working)'}


The API has been shutdown but the Json file was downloaded before the API shutdown, we will read the JSON file from the folder.

In [ ]:
## if the API was working: 
'''
data = response.json()
file_name = 'real_estate.json'

with open(file_name, 'w') as file:
    json.dump(data, file, indent = 4)
'''

In [9]:
real_estate_df = pd.read_json(r'Zuma-Realtors-Project\real_estate.json')
real_estate_df.head()

,addressLine1,city,state,zipCode,formattedAddress,bedrooms,squareFootage,yearBuilt,features,county,...,propertyTaxes,lastSalePrice,lastSaleDate,owner,id,longitude,latitude,subdivision,zoning,addressLine2
0,511 John St,Bryan,OH,43506,"511 John St, Bryan, OH 43506",3.0,1404.0,1959.0,"{'architectureType': 'Conventional', 'cooling'...",Williams,...,{'2021': {'total': 1799}},105000.0,2000-10-13T00:00:00.000Z,"{'names': ['SUSAN JO KERR'], 'mailingAddress':...","511-John-St,-Bryan,-OH-43506",-84.570163,41.469595,NaN,NaN,NaN
1,1067 Marlin Dr,Rockledge,FL,32955,"1067 Marlin Dr, Rockledge, FL 32955",3.0,2258.0,1966.0,"{'cooling': True, 'coolingType': 'Central', 'e...",Brevard,...,"{'2021': {'total': 1830}, '2022': {'total': 21...",125000.0,2019-08-13T00:00:00.000Z,"{'names': ['GYULA L KISS', 'JULIE A KISS'], 'm...","1067-Marlin-Dr,-Rockledge,-FL-32955",-80.746982,28.342440,MARLIN SUBD,R2,NaN
2,1136 Sullivan Rd,Ogden,UT,84403,"1136 Sullivan Rd, Apt 15, Ogden, UT 84403",NaN,NaN,NaN,{},Weber,...,NaN,NaN,NaN,NaN,"1136-Sullivan-Rd,-Apt-15,-Ogden,-UT-84403",-111.950330,41.204860,NaN,NaN,Apt 15
3,101 Trinity Lakes Dr,Sun City Center,FL,33573,"101 Trinity Lakes Dr, Apt 361, Sun City Center...",NaN,NaN,NaN,{},Hillsborough,...,NaN,NaN,NaN,NaN,"101-Trinity-Lakes-Dr,-Apt-361,-Sun-City-Center...",-82.355936,27.710284,NaN,NaN,Apt 361
4,7843 S Euclid Ave,Chicago,IL,60649,"7843 S Euclid Ave, Apt 1, Chicago, IL 60649",6.0,3345.0,1925.0,"{'architectureType': 'Two Family', 'cooling': ...",Cook,...,{'2020': {'total': 2527}},225000.0,2022-09-23T00:00:00.000Z,"{'names': ['78TH EUCLID LLC,'], 'mailingAddres...","7843-S-Euclid-Ave,-Apt-1,-Chicago,-IL-60649",-87.576879,41.752132,SOUTHFIELD,NaN,Apt 1


In [10]:
real_estate_df.columns

Index(['addressLine1', 'city', 'state', 'zipCode', 'formattedAddress',
       'bedrooms', 'squareFootage', 'yearBuilt', 'features', 'county',
       'assessorID', 'legalDescription', 'ownerOccupied', 'bathrooms',
       'lotSize', 'propertyType', 'taxAssessment', 'propertyTaxes',
       'lastSalePrice', 'lastSaleDate', 'owner', 'id', 'longitude', 'latitude',
       'subdivision', 'zoning', 'addressLine2'],
      dtype='object')

In [11]:
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          329 non-null    float64
 6   squareFootage     387 non-null    float64
 7   yearBuilt         342 non-null    float64
 8   features          418 non-null    object 
 9   county            500 non-null    object 
 10  assessorID        326 non-null    object 
 11  legalDescription  317 non-null    object 
 12  ownerOccupied     252 non-null    float64
 13  bathrooms         370 non-null    float64
 14  lotSize           321 non-null    float64
 15  propertyType      416 non-null    object 
 16  taxAssessment     318 non-null    object 
 1

### Data Transformation

In [17]:
# Replace missing values with appropriate replacements
real_estate_df.fillna({
    'bedrooms' : 0.0,
    'squareFootage': 0.0,
    'yearBuilt' : 0.0,
    'features' : 'Unknown',
    'assessorID' : 'Unknown',
    'legalDescription' : 'Unknown',
    'ownerOccupied' : 0.0,
    'bathrooms' : 0.0,
    'lotSize' : 0.0,
    'propertyType' : 'Unknown',
    'taxAssessment' : 'Unknown',
    'propertyTaxes' : 'Unknown',
    'lastSalePrice' : 0.0,
    'lastSaleDate' : 'Unknown',
    'zoning' : 'Unknown',
    'addressLine2' : 'Unknown'

}, inplace = True)

In [18]:
real_estate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          500 non-null    float64
 6   squareFootage     500 non-null    float64
 7   yearBuilt         500 non-null    float64
 8   features          500 non-null    object 
 9   county            500 non-null    object 
 10  assessorID        500 non-null    object 
 11  legalDescription  500 non-null    object 
 12  ownerOccupied     500 non-null    float64
 13  bathrooms         500 non-null    float64
 14  lotSize           500 non-null    float64
 15  propertyType      500 non-null    object 
 16  taxAssessment     500 non-null    object 
 1